# Percobaan Deep Learning

Input file

In [1]:
import pandas as pd
from IPython.display import display
nama_csv = 'diabetes.csv'
file = pd.read_csv(nama_csv)
display(file.head())

,Pregnancies,Glucose,Blood_Pressure,Skin_Thickness,Insulin,BMI,Diabetes_Pedigree_Function,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


preprosessing

In [2]:
import seaborn as sns
import numpy as np

columns_to_replace = ['Blood_Pressure', 'Glucose', 'Skin_Thickness', 'Insulin', 'BMI']

file[['Blood_Pressure', 'Glucose', 'Skin_Thickness', 'Insulin', 'BMI']] = file[
['Blood_Pressure','Glucose', 'Skin_Thickness', 'Insulin', 'BMI']
].replace(0, np.nan)


for col in columns_to_replace:
    file[col].fillna(file[col].mean())

file.describe()

,Pregnancies,Glucose,Blood_Pressure,Skin_Thickness,Insulin,BMI,Diabetes_Pedigree_Function,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Bagian pengolaan data

In [3]:
from sklearn.model_selection import train_test_split
X = file.drop(columns=['Outcome'])
Y = file['Outcome']
input = [8]

# Split the data into training and validation sets
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, random_state=50)

# Display the shapes of the resulting datasets
#X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

Bagian DL

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import layers

In [12]:
model = keras.Sequential([
    layers.BatchNormalization(input_shape = input),
    layers.Dense(units = 1024, activation = 'tanh'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 1024, activation = 'tanh'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 512, activation = 'relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1, activation = 'sigmoid'),
])

model.compile(
    optimizer = 'adadelta',
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy'],
)

early_stop = callbacks.EarlyStopping(
    min_delta = 0.001,
    patience = 200,
    restore_best_weights = True,
)

Plotting Val-loss and loss

In [ ]:
batch_size = 757
history = model.fit(
    X_train, Y_train,
    validation_data=(X_valid, Y_valid),
    batch_size = batch_size,
    callbacks = [early_stop],
    verbose=0, #tidak menampilkan output
    epochs = 2000
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[5:, ['loss', 'val_loss']].plot();

accuracy pllotting

In [ ]:
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

Animasi leraning rate

import matplotlib.pyplot as plt
# YOUR CODE HERE: Experiment with different values for the learning rate, batch size, and number of examples
learning_rate = 0.04
num_examples = 256

animate_sgd(
    learning_rate=learning_rate,
    batch_size=batch_size,
    num_examples=num_examples,
    # You can also change these, if you like
    steps=50, # total training steps (batches seen)
    true_w=3.0, # the slope of the data
    true_b=2.0, # the bias of the data
)

saving model 

In [ ]:
model.save('DL_test.h5')


checking model saved

In [11]:
# Load the model
loaded_model = tf.keras.models.load_model('DL_test.h5')

# Verify the model
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 8)                32        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 1024)              9216      
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 batch_normalization_2 (Batc  (None, 1024)             4